In [ ]:
import tensorflow as tf
import random
import time
import numpy as np
import glob
import concurrent.futures
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

# **TrainDataset**

In [ ]:
train_img_paths = glob.glob('../input/100-bird-species/train/*/*.jpg')
random.shuffle(train_img_paths)
train_label_names = [path.split('/')[4] for path in train_img_paths]
print('训练数据：',len(train_label_names))

In [ ]:
label_names = list(sorted(set(train_label_names)))
label_to_index = dict((name,index) for index,name in enumerate(label_names))
index_to_label = dict((v,k) for k,v in label_to_index.items())

In [ ]:
train_index_labels = [label_to_index[name] for name in train_label_names]

In [ ]:
IMSIZE = 224
def read_img(image):
    img = tf.keras.preprocessing.image.load_img(image, color_mode='rgb', target_size=(IMSIZE, IMSIZE))
    return img
def prepare_dataset(namelist, labels):
    start = time.time()
    labels = np.array(labels)
    labels = tf.convert_to_tensor(labels)
    labels = tf.expand_dims(labels,axis=-1)
    labels = tf.cast(labels, tf.int32)
    imgs = []
    with concurrent.futures.ThreadPoolExecutor(max_workers = 16) as executor:
        i = 0
        for value in executor.map(read_img, namelist):
            i+=1
            print("\rFetching: [{}/{}]".format(i, len(namelist)), end="", flush=True)
            imgs.append(value)
        imgs = np.stack(imgs)
        imgs = tf.convert_to_tensor(imgs)
    print("\nExecution time: ",time.time() - start, "s")
    return imgs, labels

In [ ]:
with tf.device('/cpu:0'):
    TRAIN_SIZE = 12000
    VAL_SIZE = 2000
    train_images, train_labels = prepare_dataset(train_img_paths[:TRAIN_SIZE], train_index_labels[:TRAIN_SIZE])
    val_images, val_labels = prepare_dataset(train_img_paths[TRAIN_SIZE:VAL_SIZE+TRAIN_SIZE], train_index_labels[TRAIN_SIZE:VAL_SIZE+TRAIN_SIZE])
print("Training Image tensor shape", train_images.shape)
print("Training Labels tensor shape", train_labels.shape)
print("Validation Image tensor shape", val_images.shape)
print("Validation Labels tensor shape", val_labels.shape)

In [ ]:
def training_map(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.1)
    #image = tf.image.random_contrast(image, 0, 1)
    image = tf.cast(image, tf.float32)
    image = image / 255
    return image, label

def validation_map(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255
    return image, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
with tf.device('/cpu:0'):
    BATCH_SIZE = 128
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(training_map,num_parallel_calls=AUTOTUNE).shuffle(TRAIN_SIZE).batch(BATCH_SIZE,drop_remainder=True)
    train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
    # del train_images, train_labels
    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(validation_map,num_parallel_calls=AUTOTUNE).batch(16,drop_remainder=True)
    val_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
    # del val_images, val_labels
    
print("Training Dataset tensor shape:", train_dataset)
print("Validation Dataset tensor shape:", val_dataset)

In [ ]:
def Create_model():
    Conv_base = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3), pooling='avg')
    Conv_base.trainable = False
    
    model = tf.keras.Sequential()
    model.add(Conv_base)
    model.add(tf.keras.layers.Dense(1024,activation='relu'))
    model.add(tf.keras.layers.Dense(250,activation='softmax'))
    
    return model

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    model = Create_model()
    optimizer = tf.keras.optimizers.Adam(0.0001)
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    # metrics
    training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
    training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('training_accuracy', dtype=tf.float32)
    validation_loss = tf.keras.metrics.Mean('validation_loss', dtype=tf.float32)
    validation_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('validation_accuracy', dtype=tf.float32)
    
    
dist_train_dataset = strategy.experimental_distribute_dataset(train_dataset)
dist_val_dataset = strategy.experimental_distribute_dataset(val_dataset)

In [ ]:
model.summary()

In [ ]:
def train_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        per_example_loss = loss_object(labels, logits)
        loss = tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZE)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)
    return loss 


@tf.function
def distributed_train_step(dist_inputs):
    per_replica_losses = strategy.run(train_step, args=(dist_inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,axis=None)

In [ ]:
def val_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
        logits = model(images, training=False)
        per_example_loss = loss_object(labels, logits)
        loss = tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZE)
    validation_loss.update_state(loss * strategy.num_replicas_in_sync)
    validation_accuracy.update_state(labels, logits)
    return loss
@tf.function
def distributed_val_step(dist_inputs):
    per_replica_losses = strategy.run(val_step, args=(dist_inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,axis=None)

In [ ]:
def train(epochs):
    for epoch in range(epochs):
              
        start = time.time()
        i = 0
        print ('\nEpoch {}/{} '.format(epoch+1, epochs))
        for data in dist_train_dataset:
            loss = distributed_train_step(data)
            
            percent = float(i+1) * 100 / len(train_dataset)
            arrow   = '-' * int(percent/100 * 10 - 1) + '>'
            spaces  = ' ' * (10 - len(arrow))
            print('\rTraining: [%s%s] %d %% - Training Loss: %f - Training ACC: %f'% (arrow, spaces, percent, training_loss.result(), training_accuracy.result()), end='', flush=True)
            i += 1
        i = 0
        print(" -", int(time.time()-start), "s", end="")
        print()
        start = time.time()
        for data in dist_val_dataset:
            loss = distributed_val_step(data)
            percent = float(i+1) * 100 / len(val_dataset)
            arrow   = '-' * int(percent/100 * 10 - 1) + '>'
            spaces  = ' ' * (10 - len(arrow))
            print('\rValidate: [%s%s] %d %% - Validation Loss: %f - Validation ACC: %f'% (arrow, spaces, percent, validation_loss.result(), validation_accuracy.result()), end='', flush=True)
            i += 1
        print(" -", int(time.time()-start), "s")
        
        # 重置 metrics
        training_loss.reset_states()
        training_accuracy.reset_states()
        validation_loss.reset_states()
        validation_accuracy.reset_states()

In [ ]:
train(30)